In [4]:
import numpy as np
import pandas as pd

# Load the dataset
file_path = '/mnt/d/Algo code/mmkp_dataset_new.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (assuming they are listed in the last row)
resource_constraints = df.iloc[-1, 3:].values.astype(int)

# Remove the constraint row from the dataframe to work with only the items
df = df.iloc[:-1]

# Simulated Annealing algorithm implementation
class SimulatedAnnealing:
    def __init__(self, df, resource_constraints, initial_temp=1000, cooling_rate=0.003):
        self.df = df
        self.resource_constraints = resource_constraints
        self.temp = initial_temp
        self.cooling_rate = cooling_rate
        self.current_solution = self.initial_solution()
        self.best_solution = self.current_solution.copy()
    
    def initial_solution(self):
        solution = []
        for group in self.df['Group'].unique():
            group_df = self.df[self.df['Group'] == group]
            min_resource_item = group_df.loc[group_df.iloc[:, 3:].sum(axis=1).idxmin()]
            solution.append(min_resource_item)
        return solution
    
    def evaluate_solution(self, solution):
        total_value = sum(item['Value'] for item in solution)
        total_resources = np.sum([item.iloc[3:].values for item in solution], axis=0)
        
        if all(total_resources <= np.array(self.resource_constraints)):
            return total_value
        else:
            return -1
    
    def neighbor_solution(self, solution):
        new_solution = solution.copy()
        group_to_change = np.random.choice(self.df['Group'].unique())
        group_df = self.df[self.df['Group'] == group_to_change]
        
        # Randomly select an item within the group that is not already in the solution
        while True:
            new_item_idx = np.random.choice(group_df.index)
            new_item = group_df.loc[new_item_idx]
            current_items_in_group = [item for item in solution if item['Group'] == group_to_change]
            if not any(new_item['Item'] == item['Item'] for item in current_items_in_group):
                break
        
        # Replace the item from the same group
        for idx, item in enumerate(new_solution):
            if item['Group'] == group_to_change:
                new_solution[idx] = new_item
                break
        
        return new_solution
    
    def accept_solution(self, candidate_value):
        current_value = self.evaluate_solution(self.current_solution)
        if candidate_value > current_value:
            return True
        else:
            delta = candidate_value - current_value
            probability = np.exp(delta / self.temp)
            return np.random.rand() < probability
    
    def run(self):
        while self.temp > 1:
            candidate_solution = self.neighbor_solution(self.current_solution)
            candidate_value = self.evaluate_solution(candidate_solution)
            
            if self.accept_solution(candidate_value):
                self.current_solution = candidate_solution
            
            if self.evaluate_solution(self.current_solution) > self.evaluate_solution(self.best_solution):
                self.best_solution = self.current_solution
            
            self.temp *= (1 - self.cooling_rate)
        
        return self.best_solution, self.evaluate_solution(self.best_solution)

# Run the simulated annealing algorithm
sa = SimulatedAnnealing(df, resource_constraints)
best_solution, best_value = sa.run()

# Output the best solution and its value
print("Best Solution:")
for item in best_solution:
    print(item['Group'], item['Item'], "Value:", item['Value'])

print("\nBest Total Value:", best_value)


Best Solution:
Group_1 Item_14 Value: 20.0
Group_2 Item_8 Value: 64.0
Group_3 Item_19 Value: 57.0
Group_4 Item_19 Value: 55.0
Group_5 Item_13 Value: 31.0
Group_6 Item_21 Value: 72.0
Group_7 Item_16 Value: 94.0
Group_8 Item_2 Value: 86.0
Group_9 Item_22 Value: 39.0
Group_10 Item_2 Value: 31.0
Group_11 Item_16 Value: 51.0
Group_12 Item_14 Value: 71.0
Group_13 Item_13 Value: 77.0
Group_14 Item_10 Value: 16.0
Group_15 Item_9 Value: 49.0
Group_16 Item_6 Value: 73.0
Group_17 Item_5 Value: 100.0
Group_18 Item_2 Value: 44.0
Group_19 Item_8 Value: 26.0
Group_20 Item_13 Value: 79.0
Group_21 Item_2 Value: 21.0
Group_22 Item_17 Value: 91.0
Group_23 Item_17 Value: 53.0
Group_24 Item_3 Value: 73.0
Group_25 Item_3 Value: 61.0
Group_26 Item_5 Value: 77.0
Group_27 Item_2 Value: 11.0
Group_28 Item_1 Value: 11.0
Group_29 Item_9 Value: 10.0
Group_30 Item_11 Value: 67.0
Group_31 Item_14 Value: 84.0
Group_32 Item_3 Value: 14.0
Group_33 Item_7 Value: 88.0
Group_34 Item_10 Value: 89.0
Group_35 Item_18 Value: 

In [24]:
import numpy as np
import pandas as pd

# Load the dataset
file_path = '/mnt/d/Algo code/mmkp_dataset_new.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (assuming they are listed in the last row)
constraint_row = df.iloc[-1, 2:]  # Extract constraints row
constraint_row = constraint_row.fillna(0).astype(int)  # Replace NaNs with 0 and convert to int
resource_constraints = constraint_row.values

# Remove the constraint row from the dataframe to work with only the items
df = df.iloc[:-1]

# Remove the zero constraint from the list
non_zero_constraints = resource_constraints[1:]  # Skip the first zero constraint

# Simulated Annealing algorithm implementation
class SimulatedAnnealing:
    def __init__(self, df, resource_constraints, initial_temp=1000, cooling_rate=0.003):
        self.df = df
        self.resource_constraints = resource_constraints
        self.temp = initial_temp
        self.cooling_rate = cooling_rate
        self.current_solution = self.initial_solution()
        self.best_solution = self.current_solution.copy()
    
    def initial_solution(self):
        solution = []
        for group in self.df['Group'].unique():
            group_df = self.df[self.df['Group'] == group]
            initial_item = group_df.sample(n=1).iloc[0]
            solution.append(initial_item)
        return solution
    
    def evaluate_solution(self, solution):
        total_value = sum(item['Value'] for item in solution)
        total_resources = np.sum([item.iloc[2:].values for item in solution], axis=0)
        
        # Remove the first zero resource in total_resources for comparison
        total_resources_filtered = total_resources[1:]
        
        # Debug print to check values
        print("Evaluating Solution:")
        print("Total Resources Used:", total_resources_filtered)
        print("Resource Constraints:", self.resource_constraints[1:])
        
        # Check if total resources used meets the constraints
        valid = all(total_resources_filtered[i] <= self.resource_constraints[1:][i] for i in range(len(self.resource_constraints) - 1))
        
        if valid:
            return total_value, total_resources_filtered
        else:
            return -1, total_resources_filtered
    
    def neighbor_solution(self, solution):
        new_solution = solution.copy()
        group_to_change = np.random.choice(self.df['Group'].unique())
        group_df = self.df[self.df['Group'] == group_to_change]
        
        # Randomly select an item within the group that is not already in the solution
        while True:
            new_item_idx = np.random.choice(group_df.index)
            new_item = group_df.loc[new_item_idx]
            if new_item['Item'] not in [item['Item'] for item in new_solution if item['Group'] == group_to_change]:
                break
        
        # Replace the item from the same group
        for idx, item in enumerate(new_solution):
            if item['Group'] == group_to_change:
                new_solution[idx] = new_item
                break
        
        return new_solution
    
    def accept_solution(self, candidate_value):
        current_value, _ = self.evaluate_solution(self.current_solution)
        if candidate_value > current_value:
            return True
        else:
            delta = candidate_value - current_value
            probability = np.exp(delta / self.temp)
            return np.random.rand() < probability
    
    def run(self):
        while self.temp > 1:
            candidate_solution = self.neighbor_solution(self.current_solution)
            candidate_value, candidate_resources = self.evaluate_solution(candidate_solution)
            
            if candidate_value != -1 and self.accept_solution(candidate_value):
                self.current_solution = candidate_solution
            
            current_value, _ = self.evaluate_solution(self.current_solution)
            best_value, _ = self.evaluate_solution(self.best_solution)
            if current_value > best_value:
                self.best_solution = self.current_solution
            
            self.temp *= (1 - self.cooling_rate)
        
        best_value, best_resources = self.evaluate_solution(self.best_solution)
        return self.best_solution, best_value, best_resources

# Run the simulated annealing algorithm
sa = SimulatedAnnealing(df, resource_constraints)
best_solution, best_value, best_resources = sa.run()

# Output the best solution and its value
print("Best Solution:")
for item in best_solution:
    print(f"Group_{item['Group']} Item_{item['Item']} Value: {item['Value']}")

print("\nBest Total Value:", best_value)
print("Total Resources Used:", best_resources)
print("Resource Constraints:", non_zero_constraints)


/tmp/ipykernel_2479/1420737700.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = constraint_row.fillna(0).astype(int)  # Replace NaNs with 0 and convert to int


Evaluating Solution:
Total Resources Used: [9 5 7 1 9 3 3 9 9 3]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [7 8 4 5 8 7 4 7 7 3]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [9 5 7 1 9 3 3 9 9 3]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [7 8 4 5 8 7 4 7 7 3]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [4 3 6 9 4 9 9 6 1 5]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [9 5 7 1 9 3 3 9 9 3]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [4 3 6 9 4 9 9 6 1 5]
Resource Constraints: [162  46 114 113  72 145 248  38 120  71]
Evaluating Solution:
Total Resources Used: [7 8 4 5 8 7 4 7 7 3]
Resource Constraints: [162  46 1